# ADAU1401 RAM / EEPROM Functional Test

https://www.analog.com/media/en/technical-documentation/data-sheets/adau1401.pdf

In [1]:
from pathfinder import Pathfinder

# abs_paths = ['']
# relative_paths = Pathfinder.relative_paths_from_abs(abs_paths)

relative_paths = [['..', '..', '..', '..', '..', '..', '已完成', 'Bridges', 'bitbucket', 'github', 'codes'],
                  ['..', '..', '..', '..', '..', 'Signal Generators', 'bitbucket', 'github', 'codes'],
                  ['..', '..', '..', '..', '..', 'Utilities', 'bitbucket', 'github', 'codes'],
                  ['..', '..', 'codes']]

Pathfinder.append_relative_paths(relative_paths)

In [2]:
import pandas as pd

#https://thispointer.com/python-pandas-how-to-display-full-dataframe-i-e-print-all-rows-columns-without-truncation/
pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth', -1)

In [3]:
from bridges.ftdi.adapters.micropython import machine
from bridges.ftdi.controllers.i2c import I2cController
from sigma.sigma_dsp.adau import ADAU1401
from sigma.bus import adapters

## Debug mode?

In [4]:
cls = ADAU1401

cls.DEBUG_MODE_SHOW_BUS_DATA = False         # whether to show bus data. 
cls.DEBUG_MODE_PRINT_REGISTER = False        # whether to print registers. 

## DSP processor

In [5]:
with_hardware_device = True  # set True if hardware is connected.

if with_hardware_device:
    
    ctrl = I2cController()
    _machine = ctrl.get_gpio()

    _i2c = ctrl.I2C()

    _pin_reset = _machine.Pin('ADBUS4', mode = machine.Pin.OUT)
    _pin_reset.high()

else:
    _i2c = _pin_reset = None  # using None for testing without actual hardware device.

bus = adapters.I2C(_i2c)

In [6]:
# bus = fx2lp.I2C(as_400KHz = False)

In [7]:
dsp = ADAU1401(bus, pin_reset = _pin_reset)

## Methods test

```
  "RAM" is the base class, which has:
 
    Program RAM (inherit RAM):
 
   Parameter RAM (inherit RAM):
 
   EEROM (inherit RAM):
messages: pack the data of EEPROM into "Messages".
read/write: inherit RAM's function.
```

### Parameter RAM

In [8]:
# read all bytes from Parameter RAM

ba_params = dsp.parameter_ram.bytes

In [9]:
# save Parameter RAM bytes to a binary file

dsp.parameter_ram.to_file('parameter_ram.bin')

In [10]:
# clear Parameter RAM

dsp.parameter_ram.clear()

In [11]:
# load Parameter RAM bytes from a binary file

dsp.parameter_ram.from_file('parameter_ram.bin');

In [12]:
# read/write_parameter: actually delegated to ADAU1401's functions.
# read/write and synchronize data with hardware.

import os
from sigma.sigma_studio.project.project_xml import *

project_xml_file_url = os.sep.join(['..', '..', 'SigmaStudio projects', 'projects', 'demo', 'demo.xml'])
ic = get_ICs(project_xml_file_url)[0]
param = ic.parameters['MultCtrlDelGrowAlg2']

dsp.parameter_ram.write_parameter(param)
dsp.parameter_ram.read_parameter(param)

b'\x00\x00\x00\x01'

In [13]:
# safe_load: safe load a single 4 bytes parameter.

dsp.parameter_ram.safe_load(param_address = param.address, data_bytes = param.bytes)

In [14]:
# safe_loads: safe load up to five parameters (= 4 x 5 = 20 bytes) at once.

dsp.parameter_ram.safe_loads(param_address = param.address ,data_bytes = param.bytes * 5)

### Program RAM

In [15]:
# read all bytes from Program RAM

ba_program = dsp.program_ram.bytes

In [16]:
# save Program RAM bytes to a binary file

dsp.program_ram.to_file('program_ram.bin')

In [17]:
# clear Program RAM

dsp.program_ram.clear()

In [18]:
# load Program RAM bytes from a binary file

dsp.program_ram.from_file('program_ram.bin');

## EEPROM

In [19]:
# read data from EEPROM into Messages

for m in dsp.eeprom.messages:
    print(m)

{'Type': 'Write', 'Type ID': 1, 'Device Address': 0, 'Sub Address': '0x081C', 'Data': b'\x00X'}
{'Type': 'No operation executed', 'Type ID': 3}
{'Type': 'No operation executed', 'Type ID': 3}
{'Type': 'No operation executed', 'Type ID': 3}
{'Type': 'No operation executed', 'Type ID': 3}
{'Type': 'No operation executed', 'Type ID': 3}
{'Type': 'No operation executed', 'Type ID': 3}
{'Type': 'No operation executed', 'Type ID': 3}
{'Type': 'No operation executed', 'Type ID': 3}
{'Type': 'No operation executed', 'Type ID': 3}
{'Type': 'No operation executed', 'Type ID': 3}
{'Type': 'No operation executed', 'Type ID': 3}
{'Type': 'No operation executed', 'Type ID': 3}
{'Type': 'No operation executed', 'Type ID': 3}
{'Type': 'No operation executed', 'Type ID': 3}
{'Type': 'No operation executed', 'Type ID': 3}
{'Type': 'No operation executed', 'Type ID': 3}
{'Type': 'No operation executed', 'Type ID': 3}
{'Type': 'No operation executed', 'Type ID': 3}
{'Type': 'Write', 'Type ID': 1, 'Device 

In [20]:
# save EEPROM bytes to a binary file

dsp.eeprom.to_file('eeprom.bin')

In [21]:
# clear EEPROM

dsp.eeprom.clear()

In [22]:
# load EEPROM bytes from a text file

file_name = 'E2Prom.hex'

with open(file_name, 'tr') as f:
    bytes_str = ''.join(f.readlines()).replace('\n', ',').replace(' ', '').split(',')
    ba = bytes(int(s, 16) for s in bytes_str if s.startswith('0x'))
    
# write bytes into EEPROM
dsp.eeprom.write(ba)

5248

In [23]:
# load EEPROM bytes from a binary file

dsp.eeprom.from_file('E2Prom.bin');